# Unit Inference in Dimtensor

## Automatic Unit Detection and Dimensional Bug Catching

This notebook demonstrates **dimtensor's automatic unit inference system**, which helps catch dimensional bugs before runtime by:

- **Variable name pattern matching**: Automatically infer units from variable names like `velocity`, `force_x`, or `energy_J`
- **Physics equation database**: 67+ equations across mechanics, electromagnetism, thermodynamics, and more
- **Constraint solver**: Infer unknown units from equations like `F = m * a`
- **CLI linting tool**: Static analysis to catch dimensional errors in your code

**Why it matters**: Dimensional errors are silent killers in scientific computing. Adding velocity to time won't raise an error in NumPy, but it's physically meaningless. Dimtensor catches these bugs automatically.

**What you'll learn**:
- How to use variable naming conventions for automatic unit detection
- How to leverage the equation database
- How to use the constraint solver to infer units
- How to integrate linting into your workflow
- Best practices and limitations

## Motivation: The Silent Bug

Consider this seemingly innocent physics calculation:

In [ ]:
import numpy as np

# Projectile motion calculation - but there's a bug!
initial_velocity = 10.0  # m/s
time = 2.0               # seconds
gravity = 9.8            # m/s^2

# Oops! We accidentally added velocity and time (wrong dimensions)
position = initial_velocity + time  # Should be: initial_velocity * time

print(f"Position: {position}")
print("No error raised! But this is physically meaningless...")

**The problem**: NumPy happily adds 10.0 and 2.0, giving 12.0. But adding velocity (m/s) to time (s) is dimensionally incorrect!

This is where **unit inference** becomes powerful. Let's see how dimtensor can catch this automatically.

## Setup

In [ ]:
# Import dimtensor and inference module
try:
    from dimtensor import DimArray, u
    from dimtensor.inference.heuristics import infer_dimension, get_matching_patterns
    from dimtensor.inference.equations import (
        EQUATION_DATABASE,
        get_equations_by_domain,
        get_equations_by_tag,
        find_equations_with_variable
    )
    from dimtensor.inference.solver import infer_units
    print("✓ All inference modules loaded successfully")
except ImportError as e:
    print(f"Error: {e}")
    print("\nMake sure dimtensor is installed with inference support:")
    print("  pip install dimtensor[all]")

## Variable Name Heuristics - Basics

Dimtensor can automatically infer units from variable names using over **150+ patterns** across multiple physics domains.

Let's start with simple examples:

In [ ]:
# Basic pattern matching
test_names = ["velocity", "force", "energy", "time", "mass", "acceleration"]

print("Variable Name Inference:\n")
for name in test_names:
    dim, confidence = infer_dimension(name)
    if dim:
        print(f"  {name:15s} -> {dim.name:12s} (confidence: {confidence:.1f})")
    else:
        print(f"  {name:15s} -> Unknown")

print("\nConfidence scores:")
print("  0.9 = Exact match (e.g., 'velocity')")
print("  0.7 = Prefix/suffix match (e.g., 'initial_velocity')")
print("  0.5 = Weak match or ambiguous")

## Exact Pattern Matches

High-confidence (0.9) exact matches across multiple physics domains:

In [ ]:
# Mechanics
mechanics = ["velocity", "acceleration", "force", "momentum", "torque", "work", "power"]

# Electromagnetism
em = ["voltage", "current", "resistance", "capacitance", "inductance", "charge"]

# Thermodynamics
thermo = ["temperature", "pressure", "entropy", "enthalpy", "heat_capacity"]

print("High-Confidence Pattern Matches:\n")

for domain, names in [("Mechanics", mechanics), ("Electromagnetism", em), ("Thermodynamics", thermo)]:
    print(f"\n{domain}:")
    for name in names:
        dim, confidence = infer_dimension(name)
        if dim:
            print(f"  {name:20s} -> {dim.name}")

## Prefix and Suffix Patterns

Dimtensor handles common naming conventions:
- **Prefixes**: `initial_`, `final_`, `max_`, `min_`, `avg_`, `total_`
- **Suffixes**: `_x`, `_y`, `_z`, `_m`, `_kg`, `_s`
- **Components**: `velocity_x`, `force_y`

In [ ]:
# Prefix examples
prefix_names = [
    "initial_velocity",
    "final_position",
    "max_force",
    "total_energy",
    "avg_power"
]

print("Prefix Pattern Matching:\n")
for name in prefix_names:
    dim, confidence = infer_dimension(name)
    if dim:
        print(f"  {name:20s} -> {dim.name:15s} (confidence: {confidence:.1f})")

# Component notation
component_names = ["velocity_x", "force_y", "acceleration_z", "momentum_x"]

print("\nComponent Pattern Matching:\n")
for name in component_names:
    dim, confidence = infer_dimension(name)
    if dim:
        print(f"  {name:20s} -> {dim.name:15s} (confidence: {confidence:.1f})")

# Unit suffixes
unit_names = ["distance_m", "energy_j", "time_s", "mass_kg"]

print("\nUnit Suffix Pattern Matching:\n")
for name in unit_names:
    dim, confidence = infer_dimension(name)
    if dim:
        print(f"  {name:20s} -> {dim.name:15s} (confidence: {confidence:.1f})")

## Ambiguous Names

Some variable names are ambiguous. For example, `v` could mean velocity or voltage. Use `get_matching_patterns()` to see all possibilities:

In [ ]:
# Ambiguous variable names
ambiguous = ["v", "p", "e", "t"]

print("Ambiguous Variable Names:\n")

for name in ambiguous:
    matches = get_matching_patterns(name)
    print(f"\n'{name}' could be:")
    if matches:
        for pattern, dimension, confidence in matches[:3]:  # Show top 3
            print(f"  {dimension.name:15s} (confidence: {confidence:.1f})")
    else:
        print("  No matches found")

print("\nRecommendation: Use descriptive names (velocity, voltage, pressure) instead of abbreviations for automatic inference.")

## Equation Database Overview

Dimtensor includes a comprehensive database of physics equations that can be used to:
1. Suggest units based on which equation you're implementing
2. Provide constraints for the solver
3. Validate dimensional consistency

Let's explore the database:

In [ ]:
print(f"Total equations in database: {len(EQUATION_DATABASE)}\n")

# Get all unique domains
domains = set()
tags = set()
for eq in EQUATION_DATABASE:
    domains.add(eq.get('domain', 'unknown'))
    tags.update(eq.get('tags', []))

print(f"Domains covered: {len(domains)}")
print("  " + ", ".join(sorted(domains)))

print(f"\nTags available: {len(tags)}")
print("  " + ", ".join(sorted(list(tags))[:10]) + ", ...")

## Searching Equations by Domain

Find all equations in a specific physics domain:

In [ ]:
# Get mechanics equations
mechanics_eqs = get_equations_by_domain("mechanics")

print(f"Mechanics equations ({len(mechanics_eqs)} total):\n")

# Display first 5 as examples
for eq in mechanics_eqs[:5]:
    print(f"  {eq['name']}")
    print(f"    Formula: {eq['formula']}")
    print(f"    Variables: {', '.join(eq['variables'].keys())}")
    if 'tags' in eq:
        print(f"    Tags: {', '.join(eq['tags'])}")
    print()

## Searching Equations by Tag

Find equations by concept (energy, power, conservation, etc.):

In [ ]:
# Find all energy-related equations
energy_eqs = get_equations_by_tag("energy")

print(f"Energy-related equations ({len(energy_eqs)} total):\n")

for eq in energy_eqs[:4]:
    print(f"  {eq['name']}: {eq['formula']}")

# Find fundamental equations
print("\n" + "="*50)
fundamental_eqs = get_equations_by_tag("fundamental")

print(f"\nFundamental equations ({len(fundamental_eqs)} total):\n")

for eq in fundamental_eqs[:4]:
    print(f"  {eq['name']}: {eq['formula']}")

## Finding Equations with Variables

Search for equations containing specific variables:

In [ ]:
# Find all equations with force (F)
force_eqs = find_equations_with_variable("F")

print(f"Equations containing 'F' (force) - showing first 5 of {len(force_eqs)}:\n")

for eq in force_eqs[:5]:
    print(f"  {eq['name']}: {eq['formula']}")
    # Show what F represents in this equation
    if 'F' in eq['variables']:
        print(f"    F = {eq['variables']['F']}")
    print()

## Constraint Solver - Basic Usage

The **constraint solver** can infer units for unknown variables in an equation based on known units.

Example: Newton's second law `F = m * a`

In [ ]:
# Define an equation with some known units
equation = "F = m * a"

# We know m is mass and a is acceleration, what is F?
known_units = {
    "m": u.kg,      # mass in kilograms
    "a": u.m / u.s**2  # acceleration in m/s^2
}

# Infer unknown units
result = infer_units(equation, known_units)

print("Constraint Solver Example:\n")
print(f"Equation: {equation}")
print(f"\nKnown:")
print(f"  m = {known_units['m']}")
print(f"  a = {known_units['a']}")
print(f"\nInferred:")
if 'F' in result:
    print(f"  F = {result['F']}")
    print(f"  (This is a Newton: kg·m/s²)")

## Constraint Solver - Complex Equations

The solver can handle more complex equations with powers, division, and multiple unknowns:

In [ ]:
# Example 1: Kinetic energy KE = 0.5 * m * v^2
equation1 = "KE = 0.5 * m * v**2"
known1 = {
    "m": u.kg,
    "v": u.m / u.s
}
result1 = infer_units(equation1, known1)

print("Example 1: Kinetic Energy\n")
print(f"Equation: {equation1}")
print(f"Known: m = kg, v = m/s")
if 'KE' in result1:
    print(f"Inferred: KE = {result1['KE']}")
    print(f"(This is a Joule: kg·m²/s²)\n")

# Example 2: Power P = W / t
equation2 = "P = W / t"
known2 = {
    "W": u.J,  # Joule (energy)
    "t": u.s   # second (time)
}
result2 = infer_units(equation2, known2)

print("\nExample 2: Power\n")
print(f"Equation: {equation2}")
print(f"Known: W = J, t = s")
if 'P' in result2:
    print(f"Inferred: P = {result2['P']}")
    print(f"(This is a Watt: J/s = kg·m²/s³)")

## Detecting Inconsistencies

The solver can detect dimensionally inconsistent equations:

In [ ]:
# Invalid equation: F = m + a (can't add mass and acceleration!)
invalid_equation = "F = m + a"
known_invalid = {
    "m": u.kg,
    "a": u.m / u.s**2
}

print("Testing dimensionally inconsistent equation:\n")
print(f"Equation: {invalid_equation}")
print(f"Known: m = kg, a = m/s²\n")

try:
    result = infer_units(invalid_equation, known_invalid)
    print(f"Result: {result}")
    print("\nNote: The solver may return a result, but it will be dimensionally inconsistent.")
    print("Always validate that your equation makes physical sense!")
except Exception as e:
    print(f"Error caught: {e}")
    print("\nThe solver detected that this equation is dimensionally invalid.")

## Real Physics Example: Projectile Motion

Let's build a projectile motion calculator and see how inference helps catch bugs:

In [ ]:
# Projectile motion equations
# x(t) = v0_x * t
# y(t) = v0_y * t - 0.5 * g * t^2

print("Projectile Motion Calculator\n")
print("Equations:")
print("  x(t) = v0_x * t")
print("  y(t) = v0_y * t - 0.5 * g * t^2\n")

# Infer units for each variable
equations = [
    ("x = v0_x * t", {"v0_x": u.m/u.s, "t": u.s}),
    ("y = v0_y * t - 0.5 * g * t**2", {"v0_y": u.m/u.s, "g": u.m/u.s**2, "t": u.s})
]

for eq, known in equations:
    result = infer_units(eq, known)
    var_name = eq.split('=')[0].strip()
    if var_name in result:
        print(f"{eq}")
        print(f"  => {var_name} has units: {result[var_name]}\n")

print("All position variables have consistent units: meters [m]")
print("This confirms our equations are dimensionally correct!")

## Bug Example #1: Dimensional Mismatch

Let's revisit our opening bug and see how dimtensor catches it:

In [ ]:
print("Bug: Adding velocity to time\n")

# Create dimtensor arrays with inferred units
from dimtensor.inference.heuristics import infer_dimension

# Infer dimensions from variable names
vel_dim, _ = infer_dimension("initial_velocity")
time_dim, _ = infer_dimension("time")

print(f"initial_velocity -> {vel_dim.name if vel_dim else 'unknown'}")
print(f"time -> {time_dim.name if time_dim else 'unknown'}\n")

# Now try to add them with dimtensor
try:
    initial_velocity = DimArray([10.0], unit=u.m/u.s)
    time = DimArray([2.0], unit=u.s)
    
    # This will raise an error!
    position = initial_velocity + time
    
except Exception as e:
    print(f"Error caught: {type(e).__name__}")
    print(f"Message: {e}")
    print("\n✓ Dimtensor caught the dimensional mismatch!")
    
# The correct operation
print("\nCorrect operation: position = initial_velocity * time")
position = initial_velocity * time
print(f"Result: {position}")

## Bug Example #2: Unit Confusion

Mixing different units without conversion is another common bug:

In [ ]:
print("Bug: Mixing kilometers and meters\n")

# Distance calculation
try:
    distance_km = DimArray([5.0], unit=u.km)
    distance_m = DimArray([100.0], unit=u.m)
    
    # These have the same dimension but different units
    # Dimtensor will convert automatically!
    total_distance = distance_km + distance_m
    
    print(f"5 km + 100 m = {total_distance}")
    print(f"             = {total_distance.to(u.m)}")
    print("\n✓ Dimtensor automatically handled unit conversion!")
    
except Exception as e:
    print(f"Error: {e}")

# Compare with plain NumPy (no unit awareness)
print("\nWith plain NumPy:")
distance_km_plain = 5.0
distance_m_plain = 100.0
total_plain = distance_km_plain + distance_m_plain
print(f"5.0 + 100.0 = {total_plain}")
print("Is this 105 km? 105 m? Who knows!")

## CLI Linting Tool - Introduction

Dimtensor includes a **CLI linting tool** that can analyze your Python code for dimensional errors without running it.

Let's create a buggy file and lint it:

In [ ]:
import tempfile
import subprocess
import os

# Create a temporary Python file with dimensional bugs
buggy_code = '''
# physics_calc.py - Contains dimensional bugs
import numpy as np

def calculate_position(velocity, time, acceleration):
    """Calculate position with bugs!"""
    # Bug 1: Adding velocity and time
    wrong_position = velocity + time
    
    # Bug 2: Adding acceleration to velocity
    wrong_velocity = velocity + acceleration
    
    # Correct calculation
    position = velocity * time + 0.5 * acceleration * time**2
    
    return position

# Variable names that inference can detect
initial_velocity = 10.0  # m/s
time = 2.0  # s
gravity = 9.8  # m/s^2

result = calculate_position(initial_velocity, time, gravity)
'''

# Write to temporary file
with tempfile.NamedTemporaryFile(mode='w', suffix='.py', delete=False) as f:
    f.write(buggy_code)
    temp_file = f.name

print("Created temporary file with dimensional bugs.")
print(f"File: {temp_file}\n")

# Try to run the linter
try:
    result = subprocess.run(
        ['dimtensor', 'lint', temp_file],
        capture_output=True,
        text=True,
        timeout=10
    )
    
    print("Linter output:")
    print(result.stdout)
    if result.stderr:
        print("Errors:")
        print(result.stderr)
        
except FileNotFoundError:
    print("Note: CLI linting tool not available in this environment.")
    print("Install with: pip install dimtensor[all]")
    print("\nThe linter would detect:")
    print("  - Line 8: Cannot add velocity (m/s) and time (s)")
    print("  - Line 11: Cannot add velocity (m/s) and acceleration (m/s²)")
finally:
    # Clean up
    os.unlink(temp_file)

## CLI Linting - Strict Mode and JSON Output

The linter supports different modes:
- **Default mode**: Reports obvious errors
- **Strict mode** (`--strict`): Also reports inference suggestions
- **JSON output** (`--json`): Machine-readable format for CI/CD integration

In [ ]:
print("CLI Linting Modes:\n")
print("Basic usage:")
print("  dimtensor lint my_script.py\n")

print("Strict mode (show all suggestions):")
print("  dimtensor lint --strict my_script.py\n")

print("JSON output (for CI/CD):")
print("  dimtensor lint --json my_script.py\n")

print("Check entire directory:")
print("  dimtensor lint src/\n")

print("Exit codes:")
print("  0 = No errors found")
print("  1 = Dimensional errors detected")
print("  2 = Linting failed (syntax errors, etc.)")

## CLI Linting - Integration

### CI/CD Integration

Add to your `.github/workflows/test.yml`:
```yaml
- name: Lint dimensional consistency
  run: dimtensor lint src/ --strict
```

### Pre-commit Hook

Add to `.pre-commit-config.yaml`:
```yaml
- repo: local
  hooks:
    - id: dimtensor-lint
      name: Dimtensor dimensional linting
      entry: dimtensor lint
      language: system
      types: [python]
```

### IDE Integration

**VS Code**: Add to `.vscode/tasks.json`:
```json
{
  "label": "Dimtensor Lint",
  "type": "shell",
  "command": "dimtensor lint ${file}",
  "problemMatcher": []
}
```

**PyCharm**: Add as External Tool in Settings → Tools → External Tools

## Confidence Levels & When to Trust Inference

Understanding confidence scores helps you know when to rely on inference vs explicit units:

In [ ]:
# Different confidence levels
test_cases = [
    ("velocity", "High confidence - exact match"),
    ("initial_velocity", "Good confidence - prefix + exact match"),
    ("v", "Low confidence - ambiguous (velocity or voltage?)"),
    ("myvar", "No inference - generic name")
]

print("Confidence Level Guide:\n")

for name, description in test_cases:
    dim, confidence = infer_dimension(name)
    print(f"{name:20s} (confidence: {confidence:.1f})")
    print(f"  {description}")
    if dim:
        print(f"  Inferred: {dim.name}")
    else:
        print(f"  Inferred: None")
    print()

print("\nGuidelines:")
print("  Confidence ≥ 0.9: Safe to use in production")
print("  Confidence 0.7-0.9: Good for prototyping, verify in production")
print("  Confidence < 0.7: Use explicit units instead")
print("  No match: Always use explicit units")

## Best Practices

### When to Rely on Inference
✅ **DO use inference for:**
- Rapid prototyping and exploration
- Educational notebooks and tutorials
- Code with clear, standard variable names
- Internal calculations with well-established physics

❌ **DON'T rely on inference for:**
- Production APIs and library interfaces (use explicit units)
- Code with ambiguous or abbreviated variable names
- Critical safety or financial calculations
- Non-standard or domain-specific units

### Naming Conventions for Best Inference
```python
# Good - clear and standard
velocity, acceleration, force, energy, power
initial_velocity, final_position, max_force
velocity_x, force_y, acceleration_z

# Avoid - ambiguous or unclear
v, a, f, e, p  # Too short, ambiguous
myvar, temp1, result  # Generic, no physics meaning
vel_final_after_collision  # Too complex for pattern matching
```

### Hybrid Approach (Recommended)
```python
# Use inference for internal calculations
def calculate_kinetic_energy(mass, velocity):
    # Inference helps validate internal logic
    energy = 0.5 * mass * velocity**2
    return energy

# But use explicit units at API boundaries
m = DimArray([1.0], unit=u.kg)
v = DimArray([10.0], unit=u.m/u.s)
ke = calculate_kinetic_energy(m, v)
```

## Limitations & Caveats

Understanding the limitations helps avoid pitfalls:

In [ ]:
print("Inference Limitations:\n")

print("1. AMBIGUOUS NAMES")
print("   Problem: 'v' could be velocity or voltage")
print("   Solution: Use descriptive names\n")

print("2. NON-STANDARD NAMING")
name = "vel_after_bounce_2"
dim, conf = infer_dimension(name)
print(f"   Problem: '{name}' -> {dim.name if dim else 'None'} (confidence: {conf})")
print("   Solution: Simplify to 'velocity_2' or 'final_velocity'\n")

print("3. DOMAIN-SPECIFIC UNITS")
print("   Problem: Astronomy units (parsec, solar mass) not in pattern database")
print("   Solution: Use explicit units for specialized domains\n")

print("4. COMPLEX COUPLED EQUATIONS")
print("   Problem: Solver may struggle with systems of 10+ coupled equations")
print("   Solution: Break into smaller sub-problems\n")

print("5. UNITLESS NUMBERS")
print("   Problem: Constants like 0.5, pi, 2.0 have no units")
print("   Solution: This is correct! Dimensionless constants are fine\n")

print("6. CONFLICTING CONSTRAINTS")
print("   Problem: Over-specified systems may have no solution")
print("   Solution: Check equation consistency first\n")

print("Remember: Inference is a tool to help you, not a replacement for understanding the physics!")

## Summary & Next Steps

### What You Learned

In this notebook, you learned how to:
1. ✓ Use variable name heuristics for automatic unit detection
2. ✓ Search the physics equation database
3. ✓ Use the constraint solver to infer units from equations
4. ✓ Integrate the CLI linting tool into your workflow
5. ✓ Catch dimensional bugs before runtime
6. ✓ Follow best practices for reliable inference

### Key Takeaways

- **150+ patterns** across mechanics, EM, thermo, quantum, relativity
- **67+ equations** in the database for reference and solving
- **Confidence scores** help you know when to trust inference
- **CLI linting** enables static analysis in CI/CD pipelines
- **Best practice**: Use inference for development, explicit units for production APIs

### Next Steps

1. Try using inference in your own physics code
2. Add `dimtensor lint` to your CI/CD pipeline
3. Contribute new patterns or equations to the database
4. Explore other dimtensor features:
   - PyTorch integration (DimTensor with autograd)
   - JAX integration (JIT compilation with units)
   - Physical constants (CODATA 2022)
   - Serialization (JSON, HDF5, Pandas)

### Further Reading

- [Dimtensor Documentation](https://dimtensor.readthedocs.io)
- [Inference Module API](https://dimtensor.readthedocs.io/en/latest/api/inference.html)
- [CLI Linting Guide](https://dimtensor.readthedocs.io/en/latest/guide/linting.html)
- [Contributing Guide](https://github.com/yourusername/dimtensor/blob/main/CONTRIBUTING.md)

**Happy dimensional programming! 🚀**